## Librerias

In [17]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
import seaborn as sns
from scipy.stats import chi2_contingency
import scipy.stats as ss
import os
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer

import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBClassifier

from sklearn.preprocessing import FunctionTransformer


## Funciones

In [4]:
def agrupar_categorias_pequenas(df, columna, umbral=100):
    # Contar la frecuencia de cada categoría
    conteo_categorias = df[columna].value_counts()
    
    # Identificar categorías con observaciones por debajo del umbral
    categorias_pequenas = conteo_categorias[conteo_categorias < umbral].index
    
    # Mantener el mismo tipo de las categorías originales
    tipo_original = df[columna].dtype.type
    
    # Reemplazar esas categorías con 'Otros', asegurando el mismo tipo de dato
    df[columna] = df[columna].apply(lambda x: tipo_original('Otros') if x in categorias_pequenas else x)
    
    return df

def cramers_v(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix, correction=False)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    if min(r, k) == 1:
        return np.nan  # Evita dividir por cero
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

def expand_list(df, column):
    # Crear un conjunto de categorías
    categories = set()
    
    # Recorrer todas las listas y agregar cada categoría única al conjunto
    for row in df[column]:
        if pd.notna(row):
            # Verificar si el valor es una lista; si es string, se convierte en lista
            if isinstance(row, str):
                try:
                    row_list = eval(row)  # Transforma la string en lista
                except:
                    row_list = [row]  # Si no es lista, lo trata como valor único
            else:
                row_list = row if isinstance(row, list) else [row]
            
            # Añadir los elementos de la lista a las categorías
            categories.update(map(lambda item: f"{item}_{column}", map(str, row_list)))  # Convertir todos los elementos a string con el formato correcto

    # Crear un DataFrame temporal para almacenar las nuevas columnas
    new_columns = pd.DataFrame(index=df.index)

    # Llenar el DataFrame temporal con las nuevas columnas
    for category in categories:
        new_columns[category] = df[column].apply(lambda x: 1 if pd.notna(x) and str(category.split('_')[0]) in str(x) else 0)

    # Concatenar el nuevo DataFrame con las columnas originales
    df = pd.concat([df.drop(columns=[column]), new_columns], axis=1)

    return categories, df


def categorizar_hora(hora):
        hora = int(hora.split(':')[0])
        if 0 <= hora < 12:
            return 'mañana'
        elif 12 <= hora < 18:
            return 'tarde'
        else:
            return 'noche'

def convertir_auction_time(df):
    
    df['auction_time'] = pd.to_datetime(df['auction_time'], unit='s').dt.strftime('%H:%M:%S')
    
    # Crear una nueva columna con las categorías de tiempo
    df['parte_del_dia'] = df['auction_time'].apply(categorizar_hora)
    
    # Crear columnas binarias para cada parte del día
    df['mañana'] = (df['parte_del_dia'] == 'mañana').astype(int)
    df['tarde'] = (df['parte_del_dia'] == 'tarde').astype(int)
    df['noche'] = (df['parte_del_dia'] == 'noche').astype(int)
    
    # Opcional: Eliminar la columna 'parte_del_dia' si no se necesita
    df.drop(['parte_del_dia', 'auction_time'], axis=1, inplace=True)

    return ['mañana', 'tarde', 'noche'], df

def imputar_nan_como_otros(df):
    for columna in df.columns:
        if df[columna].isnull().any():  # Verificar si hay NaN en la columna
            tipo_original = df[columna].dtype  # Guardar el tipo original de la columna
            # Imputar 'Otros' y asegurarse de que el tipo de dato se mantiene
            df[columna] = df[columna].fillna(tipo_original.type('Otros'))
    return df

def data_cleaning(df):
    all_categories = set()

    categorical_features = [col for col in df.select_dtypes(include=['object']).columns.tolist() if col not in ['auction_list_0', 'action_list_1', 'action_list_2', 'auction_time']]

    categories, df = convertir_auction_time(df)
    all_categories.update(categories)

    categories, df = expand_list(df, 'auction_list_0')
    all_categories.update(categories)

    categories, df = expand_list(df, 'action_list_1')
    all_categories.update(categories)

    categories, df = expand_list(df, 'action_list_2')
    all_categories.update(categories)

    for col in categorical_features:
        df = agrupar_categorias_pequenas(df, col, umbral=100) 

    return all_categories, df

def ajustar_columnas_test(train_df, test_df):
    """
    Asegura que las columnas del conjunto de test coincidan con las del conjunto de entrenamiento,
    con la excepción de que 'Label' no se puede borrar de train y 'id' no se puede borrar de test.
    
    Si faltan columnas en el conjunto de test, se agregan con valor 0.
    Si sobran columnas en el conjunto de test que no están en el de entrenamiento, se eliminan.
    
    Parameters:
    - train_df: DataFrame del conjunto de entrenamiento (con las columnas transformadas)
    - test_df: DataFrame del conjunto de test (sin transformar)
    
    Returns:
    - test_df: DataFrame del conjunto de test con las columnas ajustadas.
    """
    # Encontrar las columnas que están en train pero no en test, excepto 'Label'
    missing_cols = set(train_df.columns) - set(test_df.columns) - {'Label'}

    # Encontrar las columnas que están en test pero no en train, excepto 'id'
    extra_cols = set(test_df.columns) - set(train_df.columns) - {'id'}

    # Agregar las columnas faltantes en el conjunto de test con valor 0
    for col in missing_cols:
        test_df[col] = 0

    # Eliminar las columnas extra del conjunto de test, excepto 'id'
    test_df = test_df.drop(columns=extra_cols)

    # Asegurarse de que las columnas estén en el mismo orden que en el conjunto de entrenamiento
    test_df = test_df[train_df.columns.difference(['Label']).tolist() + ['id']]

    return test_df



## Data

In [3]:
# train_data_21 = pd.read_csv("data/ctr_21.csv")
# train_data_20 = pd.read_csv("data/ctr_20.csv")
# train_data_19 = pd.read_csv("data/ctr_19.csv")
# train_data_18 = pd.read_csv("data/ctr_18.csv")
# train_data_17 = pd.read_csv("data/ctr_17.csv")
# train_data_16 = pd.read_csv("data/ctr_16.csv")
# train_data_15 = pd.read_csv("data/ctr_15.csv")
# train_data_combined = pd.concat([train_data_21, train_data_20, train_data_19, train_data_18, train_data_17, train_data_16, train_data_15], axis=0)
# train_data_combined = pd.concat([train_data_21, train_data_20], axis=0)
train_data_combined = pd.read_csv("data/ctr_21.csv")


In [4]:
random_state = 43992294

In [ ]:
# Reducir el conjunto de datos mientras se mantiene la distribución de 'label'
train_data_combined, _ = train_test_split(train_data_combined, train_size=100000, stratify=train_data_combined['Label'], random_state=42)

# Verificar la distribución en el conjunto reducido
print(train_data_combined['Label'].value_counts(normalize=True))

## Caracteristicas principales

In [ ]:
# Imprimir la cantidad de filas del dataset combinado
print(f"Cantidad de filas en el dataset combinado: {train_data_combined.shape[0]}")

# Ver porcentaje de clics vs no clics en la columna Label
label_counts = train_data_combined['Label'].value_counts(normalize=True) * 100
print("\nPorcentaje de clics (1) y no clics (0):")
print(label_counts)

# Cantidad de clics (1) y no clics (0)
label_counts_abs = train_data_combined['Label'].value_counts()
print("\nCantidad de clics (1) y no clics (0):")
print(label_counts_abs)

In [ ]:
# Resumen estadístico de las características numéricas
print(train_data_combined.describe())

# Resumen de las características categóricas
print(train_data_combined.describe(include='object'))

In [ ]:
# Ver el porcentaje de valores faltantes por columna
missing_data = train_data_combined.isnull().mean() * 100
print(missing_data[missing_data > 0].sort_values(ascending=False))

In [5]:
# Dividir los datos en numéricos y categóricos
numeric_features = train_data_combined.select_dtypes(include=['number']).columns.tolist()

categorical_features = train_data_combined.select_dtypes(include=['object']).columns.tolist()

## Columna por columna

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in train_data_combined.columns:
    unique_values = train_data_combined[column].unique()
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos y la cantidad de veces que aparecen
for column in train_data_combined.columns:
    print(f"Columna: {column}")
    print(f"Valores únicos y su frecuencia:")
    print(train_data_combined[column].value_counts())
    print("-" * 50)  # Separador entre columnas

In [ ]:
# Inicializar un conjunto vacío para almacenar los entity ids únicos
unique_entity_ids_2 = set()

# Recorrer cada fila de la columna 'action_list_2'
for lista in train_data_combined['action_list_2'].dropna():
    # Convertir el string de la lista a una lista real y agregar cada valor al conjunto
    unique_entity_ids_2.update(eval(lista))  # eval transforma la string en lista si es necesario

# Convertir el conjunto en una lista ordenada
unique_entity_ids_2 = sorted(unique_entity_ids_2)

# Mostrar los entity ids únicos
print(unique_entity_ids_2)
print(len(unique_entity_ids_2))



In [ ]:
# Inicializar un conjunto vacío para almacenar los entity ids únicos
unique_entity_ids = set()

# Recorrer cada fila de la columna 'action_list_1'
for lista in train_data_combined['action_list_1'].dropna():
    # Convertir el string de la lista a una lista real y agregar cada valor al conjunto
    unique_entity_ids.update(eval(lista))  # eval transforma la string en lista si es necesario

# Convertir el conjunto en una lista ordenada
unique_entity_ids = sorted(unique_entity_ids)

# Mostrar los entity ids únicos
print(unique_entity_ids)
print(len(unique_entity_ids))

In [ ]:
# Calcular la cantidad de valores NaN en cada columna
nan_counts = train_data_combined.isna().sum()

# Filtrar columnas que tienen al menos un NaN
nan_counts_filtered = nan_counts[nan_counts > 0]

# Mostrar las columnas con valores NaN de mayor a menor
print(nan_counts_filtered.sort_values(ascending=False))

## Data cleaning

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in train_data_combined.columns:
    unique_values = train_data_combined[column].unique()
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

In [5]:
categories, train_data_combined = data_cleaning(train_data_combined)

In [6]:
# Guardar el dataset modificado como CSV antes de aplicar One-Hot Encoding
train_data_combined.to_csv('train_data_cleaned.csv', index=False)

In [ ]:
train_data_combined.columns.tolist()

In [ ]:
# Iterar sobre cada columna y mostrar los valores únicos
for column in train_data_combined.columns:
    unique_values = train_data_combined[column].unique()  # Obtener los valores únicos para cada columna
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

In [ ]:
train_data_combined.isnull().sum()[train_data_combined.isnull().sum() > 0]

# Encodings

## Target Encoding

In [ ]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Verificar la proporción de clases en el conjunto de entrenamiento y validación
print("Proporción en el conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True))

print("Proporción en el conjunto de validación:")
print(y_val.value_counts(normalize=True))


In [ ]:
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

# Crear el Target Encoder para las columnas categóricas
target_encoder = ce.TargetEncoder(cols=categorical_features)

# Ajustar y transformar los datos de entrenamiento
X_train_encoded = target_encoder.fit_transform(X_train, y_train)

# Transformar los datos de validación
X_val_encoded = target_encoder.transform(X_val)

# Crear un imputador que llene los valores NaN con la media de cada columna
imputer = SimpleImputer(strategy='mean')

# Aplicar el imputador a los datos de entrenamiento
X_train_encoded = imputer.fit_transform(X_train_encoded)

# También aplica el imputador al conjunto de validación
X_val_encoded = imputer.transform(X_val_encoded)

# Convertir los resultados a DataFrame y asignar las columnas originales
X_train_encoded = pd.DataFrame(X_train_encoded, columns=X_train.columns)
X_val_encoded = pd.DataFrame(X_val_encoded, columns=X_val.columns)

# Ver los primeros datos
print(X_train_encoded.head())
print(X_val_encoded.head())

In [ ]:
# Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

# Eliminar las columnas no deseadas
test_data = test_data.drop(columns=columns_to_drop)

test_data_encoded = target_encoder.transform(test_data.drop(columns=["id"]))
test_data_encoded = imputer.transform(test_data_encoded)  # Imputar los valores faltantes en el set de testeo

# Convertir los resultados a DataFrame y asignar las columnas originales
test_data_encoded = pd.DataFrame(test_data_encoded, columns=test_data.drop(columns=["id"]).columns)

# comparar tamaño de columnas
print(X_train_encoded.columns)
print(test_data_encoded.columns)

## One Hot Encoding

In [8]:
random_state = 43992294

In [5]:
train_data_combined = pd.read_csv('train_data_cleaned.csv')

In [6]:
categories = {'-2560_action_list_2', '-2606_action_list_1', '-5469_action_list_2', '-5470_action_list_2', '-5471_action_list_2', '-5559_action_list_1', '-5559_action_list_2', '-5560_action_list_2', '-5576_action_list_2', '-5577_action_list_1', '-5577_action_list_2', '-5578_action_list_1', '-5578_action_list_2', '-5579_action_list_1', '-5579_action_list_2', '-5603_action_list_2', '-5604_action_list_2', '-5605_action_list_2', '-5613_action_list_2', '-5736_action_list_1', '-5902_action_list_2', '-6118_action_list_1', '-6118_action_list_2', '-6119_action_list_1', '-6119_action_list_2', '-6125_action_list_2', '-6217_action_list_2', '-6218_action_list_2', '-6219_action_list_2', '-6220_action_list_2', '-6223_action_list_2', '-6224_action_list_2', '-6226_action_list_2', '-6309_action_list_2', '-6451_action_list_1', '-6454_action_list_1', '-6454_action_list_2', '-6543_action_list_2', '-6544_action_list_2', '-6547_action_list_2', '-6548_action_list_2', '-6613_action_list_1', '-6614_action_list_1', '-6615_action_list_1', '-6615_action_list_2', '-6616_action_list_1', '-6617_action_list_1', '-6617_action_list_2', '-6618_action_list_1', '-6618_action_list_2', '-6620_action_list_2', '-6621_action_list_2', '-6770_action_list_2', '-6774_action_list_2', '-6775_action_list_2', '-6779_action_list_1', '-6780_action_list_1', '-6780_action_list_2', '-6800_action_list_2', '-6823_action_list_1', '-6823_action_list_2', '-6824_action_list_1', '-6848_action_list_2', '-6849_action_list_2', '-6850_action_list_2', '-6865_action_list_2', '-6866_action_list_2', '-6867_action_list_2', '-6871_action_list_1', '-6871_action_list_2', '-6874_action_list_1', '-6875_action_list_2', '-6876_action_list_2', '-6902_action_list_1', '-6903_action_list_1', '-6904_action_list_2', '-6905_action_list_2', '-6929_action_list_2', '-6938_action_list_2', '-6946_action_list_2', '-7033_action_list_1', '-7111_action_list_2', '-7112_action_list_2', '-7126_action_list_2', '-7127_action_list_2', '-7143_action_list_2', '-7190_action_list_1', '-7190_action_list_2', '-7194_action_list_1', '-7195_action_list_1', '-7195_action_list_2', '-7196_action_list_2', '-7199_action_list_1', '-7263_action_list_2', '-7264_action_list_2', '-7265_action_list_2', '2560_action_list_2', '2606_action_list_1', '5469_action_list_2', '5470_action_list_2', '5471_action_list_2', '5559_action_list_1', '5559_action_list_2', '5560_action_list_2', '5576_action_list_2', '5577_action_list_1', '5577_action_list_2', '5578_action_list_1', '5578_action_list_2', '5579_action_list_1', '5579_action_list_2', '5603_action_list_2', '5604_action_list_2', '5605_action_list_2', '5613_action_list_2', '5736_action_list_1', '5902_action_list_2', '6118_action_list_1', '6118_action_list_2', '6119_action_list_1', '6119_action_list_2', '6125_action_list_2', '6217_action_list_2', '6218_action_list_2', '6219_action_list_2', '6220_action_list_2', '6223_action_list_2', '6224_action_list_2', '6226_action_list_2', '6309_action_list_2', '6451_action_list_1', '6451_action_list_2', '6454_action_list_1', '6454_action_list_2', '6543_action_list_2', '6544_action_list_2', '6547_action_list_2', '6548_action_list_2', '6614_action_list_1', '6615_action_list_1', '6615_action_list_2', '6616_action_list_1', '6616_action_list_2', '6617_action_list_1', '6618_action_list_1', '6618_action_list_2', '6620_action_list_2', '6621_action_list_2', '6770_action_list_2', '6774_action_list_2', '6775_action_list_2', '6779_action_list_1', '6780_action_list_1', '6780_action_list_2', '6800_action_list_2', '6823_action_list_1', '6823_action_list_2', '6824_action_list_1', '6848_action_list_2', '6849_action_list_2', '6850_action_list_2', '6865_action_list_2', '6866_action_list_2', '6867_action_list_2', '6871_action_list_1', '6871_action_list_2', '6874_action_list_1', '6875_action_list_2', '6876_action_list_2', '6902_action_list_1', '6903_action_list_1', '6904_action_list_2', '6905_action_list_2', '6929_action_list_2', '6938_action_list_2', '6946_action_list_2', '7033_action_list_1', '7111_action_list_2', '7112_action_list_2', '7126_action_list_2', '7127_action_list_2', '7143_action_list_2', '7190_action_list_1', '7190_action_list_2', '7194_action_list_1', '7195_action_list_1', '7195_action_list_2', '7196_action_list_2', '7199_action_list_1', '7263_action_list_2', '7264_action_list_2', '7265_action_list_2', 'IAB-5_auction_list_0', 'IAB1-1_auction_list_0', 'IAB1-2_auction_list_0', 'IAB1-3_auction_list_0', 'IAB1-4_auction_list_0', 'IAB1-5_auction_list_0', 'IAB1-6_auction_list_0', 'IAB1-7_auction_list_0', 'IAB10-1_auction_list_0', 'IAB10-4_auction_list_0', 'IAB10_auction_list_0', 'IAB11-2_auction_list_0', 'IAB11-4_auction_list_0', 'IAB11_auction_list_0', 'IAB12-1_auction_list_0', 'IAB12-2_auction_list_0', 'IAB12-3_auction_list_0', 'IAB12_auction_list_0', 'IAB13-7_auction_list_0', 'IAB13_auction_list_0', 'IAB14-1_auction_list_0', 'IAB14-3_auction_list_0', 'IAB14-7_auction_list_0', 'IAB14_auction_list_0', 'IAB15-10_auction_list_0', 'IAB15-6_auction_list_0', 'IAB15_auction_list_0', 'IAB16-2_auction_list_0', 'IAB16-4_auction_list_0', 'IAB16-6_auction_list_0', 'IAB16_auction_list_0', 'IAB17-12_auction_list_0', 'IAB17-13_auction_list_0', 'IAB17-1_auction_list_0', 'IAB17-27_auction_list_0', 'IAB17-39_auction_list_0', 'IAB17-44_auction_list_0', 'IAB17_auction_list_0', 'IAB18-3_auction_list_0', 'IAB18_auction_list_0', 'IAB19-10_auction_list_0', 'IAB19-11_auction_list_0', 'IAB19-12_auction_list_0', 'IAB19-13_auction_list_0', 'IAB19-14_auction_list_0', 'IAB19-15_auction_list_0', 'IAB19-16_auction_list_0', 'IAB19-17_auction_list_0', 'IAB19-18_auction_list_0', 'IAB19-19_auction_list_0', 'IAB19-1_auction_list_0', 'IAB19-20_auction_list_0', 'IAB19-21_auction_list_0', 'IAB19-22_auction_list_0', 'IAB19-23_auction_list_0', 'IAB19-24_auction_list_0', 'IAB19-25_auction_list_0', 'IAB19-26_auction_list_0', 'IAB19-27_auction_list_0', 'IAB19-28_auction_list_0', 'IAB19-29_auction_list_0', 'IAB19-2_auction_list_0', 'IAB19-30_auction_list_0', 'IAB19-31_auction_list_0', 'IAB19-32_auction_list_0', 'IAB19-33_auction_list_0', 'IAB19-34_auction_list_0', 'IAB19-35_auction_list_0', 'IAB19-36_auction_list_0', 'IAB19-3_auction_list_0', 'IAB19-4_auction_list_0', 'IAB19-5_auction_list_0', 'IAB19-6_auction_list_0', 'IAB19-7_auction_list_0', 'IAB19-8_auction_list_0', 'IAB19_auction_list_0', 'IAB1_auction_list_0', 'IAB2-12_auction_list_0', 'IAB2-14_auction_list_0', 'IAB2-17_auction_list_0', 'IAB2-4_auction_list_0', 'IAB2-8_auction_list_0', 'IAB20-1_auction_list_0', 'IAB20-3_auction_list_0', 'IAB20_auction_list_0', 'IAB21_auction_list_0', 'IAB22_auction_list_0', 'IAB24_auction_list_0', 'IAB2_auction_list_0', 'IAB3-3_auction_list_0', 'IAB3-5_auction_list_0', 'IAB3_auction_list_0', 'IAB4-4_auction_list_0', 'IAB4-5_auction_list_0', 'IAB4_auction_list_0', 'IAB5-1_auction_list_0', 'IAB5-6_auction_list_0', 'IAB5-7_auction_list_0', 'IAB5-8_auction_list_0', 'IAB5-9_auction_list_0', 'IAB5_auction_list_0', 'IAB6-3_auction_list_0', 'IAB6-4_auction_list_0', 'IAB6-5_auction_list_0', 'IAB6-6_auction_list_0', 'IAB6_auction_list_0', 'IAB7-10_auction_list_0', 'IAB7-11_auction_list_0', 'IAB7-12_auction_list_0', 'IAB7-13_auction_list_0', 'IAB7-14_auction_list_0', 'IAB7-15_auction_list_0', 'IAB7-16_auction_list_0', 'IAB7-17_auction_list_0', 'IAB7-18_auction_list_0', 'IAB7-19_auction_list_0', 'IAB7-1_auction_list_0', 'IAB7-20_auction_list_0', 'IAB7-21_auction_list_0', 'IAB7-22_auction_list_0', 'IAB7-23_auction_list_0', 'IAB7-24_auction_list_0', 'IAB7-25_auction_list_0', 'IAB7-26_auction_list_0', 'IAB7-27_auction_list_0', 'IAB7-28_auction_list_0', 'IAB7-29_auction_list_0', 'IAB7-2_auction_list_0', 'IAB7-30_auction_list_0', 'IAB7-31_auction_list_0', 'IAB7-32_auction_list_0', 'IAB7-33_auction_list_0', 'IAB7-34_auction_list_0', 'IAB7-35_auction_list_0', 'IAB7-36_auction_list_0', 'IAB7-37_auction_list_0', 'IAB7-38_auction_list_0', 'IAB7-39_auction_list_0', 'IAB7-3_auction_list_0', 'IAB7-40_auction_list_0', 'IAB7-41_auction_list_0', 'IAB7-42_auction_list_0', 'IAB7-43_auction_list_0', 'IAB7-44_auction_list_0', 'IAB7-45_auction_list_0', 'IAB7-4_auction_list_0', 'IAB7-5_auction_list_0', 'IAB7-6_auction_list_0', 'IAB7-7_auction_list_0', 'IAB7-8_auction_list_0', 'IAB7-9_auction_list_0', 'IAB7_auction_list_0', 'IAB8-6_auction_list_0', 'IAB8_auction_list_0', 'IAB9-10_auction_list_0', 'IAB9-11_auction_list_0', 'IAB9-12_auction_list_0', 'IAB9-13_auction_list_0', 'IAB9-14_auction_list_0', 'IAB9-15_auction_list_0', 'IAB9-16_auction_list_0', 'IAB9-17_auction_list_0', 'IAB9-18_auction_list_0', 'IAB9-19_auction_list_0', 'IAB9-1_auction_list_0', 'IAB9-20_auction_list_0', 'IAB9-21_auction_list_0', 'IAB9-22_auction_list_0', 'IAB9-23_auction_list_0', 'IAB9-24_auction_list_0', 'IAB9-25_auction_list_0', 'IAB9-26_auction_list_0', 'IAB9-27_auction_list_0', 'IAB9-28_auction_list_0', 'IAB9-29_auction_list_0', 'IAB9-2_auction_list_0', 'IAB9-30_auction_list_0', 'IAB9-31_auction_list_0', 'IAB9-3_auction_list_0', 'IAB9-4_auction_list_0', 'IAB9-5_auction_list_0', 'IAB9-6_auction_list_0', 'IAB9-7_auction_list_0', 'IAB9-8_auction_list_0', 'IAB9-9_auction_list_0', 'IAB9_auction_list_0', 'books_auction_list_0', 'business_auction_list_0', 'education_auction_list_0', 'entertainment_auction_list_0', 'finance_auction_list_0', 'games_auction_list_0', 'healthcare_and_fitness_auction_list_0', 'lifestyle_auction_list_0', 'mañana', 'medical_auction_list_0', 'music_auction_list_0', 'navigation_auction_list_0', 'news_auction_list_0', 'noche', 'photography_auction_list_0', 'productivity_auction_list_0', 'reference_auction_list_0', 'social_networking_auction_list_0', 'sports_auction_list_0', 'tarde', 'travel_auction_list_0', 'utilities_auction_list_0', 'weather_auction_list_0'}

In [ ]:
train_data_combined.columns.tolist()

In [9]:
# Dividir los datos en entrenamiento y validación con estratificación
X_train, X_val, y_train, y_val = train_test_split(
    train_data_combined.drop(columns='Label'),  # Características
    train_data_combined['Label'],               # Variable objetivo
    test_size=0.2,                              # 20% para validación
    stratify=train_data_combined['Label'],      # Estratificación basada en la variable objetivo
    random_state=random_state                   # Semilla para reproducibilidad
)

# Asegurarse de que X_train y X_val sean dataframes de pandas
X_train = pd.DataFrame(X_train)
X_val = pd.DataFrame(X_val)

# Asegurarse de que y_train y y_val sean series de pandas
y_train = pd.Series(y_train)
y_val = pd.Series(y_val)

In [15]:
pd.set_option('display.max_rows', None)
for column in X_train.columns:
    # Obtén los tipos de datos únicos por columna
    unique_types = X_train[column].apply(type).unique()
    if len(unique_types) > 1:
        print(f"Columna: {column}, Tipos: {unique_types}")

Columna: action_categorical_6, Tipos: [<class 'str'> <class 'int'>]
Columna: auction_boolean_0, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_boolean_1, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_boolean_2, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_categorical_0, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_categorical_11, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_categorical_12, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_categorical_2, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_categorical_3, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_categorical_4, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_categorical_6, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_categorical_7, Tipos: [<class 'str'> <class 'float'>]
Columna: auction_categorical_9, Tipos: [<class 'float'> <class 'str'>]
Columna: creative_categorical_12, Tipos: [<class 'float'> <class 'str'>]
Columna: creative

In [18]:
# Definir las columnas categóricas y numéricas
categorical_features_to_encode = [col for col in X_train.select_dtypes(include=['object', 'int', 'float']).columns if col not in categories]
numeric_features = X_train.select_dtypes(include=['number']).columns.tolist()

# Crear el preprocesador para manejar NaNs y aplicar One-Hot Encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Desconocido')),
            ('convert_to_string', FunctionTransformer(lambda x: x.astype(str), validate=False)),  # Convertir todos los valores a cadena
            ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
        ]), categorical_features_to_encode)
    ],
    remainder='passthrough'  # Mantener el resto de las columnas como están
)

# Aplicar el pipeline de preprocesamiento a los datos de entrenamiento
X_train_encoded = preprocessor.fit_transform(X_train)
# Aplicar el pipeline al conjunto de validación
X_val_encoded = preprocessor.transform(X_val)

# Convertir los resultados a DataFrame si es necesario
X_train_encoded = pd.DataFrame(X_train_encoded, columns=preprocessor.get_feature_names_out())
X_val_encoded = pd.DataFrame(X_val_encoded, columns=preprocessor.get_feature_names_out())

: 

In [ ]:
for column in X_train_encoded.columns:
    unique_values = X_train_encoded[column].unique()  # Obtener los valores únicos
    print(f"Columna: {column}")
    print(f"Valores únicos ({len(unique_values)}): {unique_values}")
    print("-" * 50)  # Separador entre columnas

In [23]:
# Paso 1: Cargar los datos de testeo
test_data = pd.read_csv("data/ctr_test.csv")

# Paso 2: Modificar la columna auction_list_0 (expansión de la lista en múltiples columnas)
categories2, test_data = data_cleaning(test_data)

In [25]:
# Paso 3: Ajustar las columnas del conjunto de testeo
test_data = ajustar_columnas_test(X_train, test_data)

In [26]:
# Paso 4: Aplicar el preprocesador al conjunto de testeo
test_data_encoded = preprocessor.transform(test_data)

# Convertir los resultados a DataFrame si es necesario
test_data_encoded = pd.DataFrame(test_data_encoded, columns=preprocessor.get_feature_names_out())


In [27]:
test_data_encoded.to_csv('test_data_cleaned.csv', index=False)

In [ ]:
test_data_encoded = pd.read_csv('test_data_cleaned.csv')

test_data = pd.read_csv("data/ctr_test.csv")

print(f"Columnas en el conjunto de entrenamiento: {X_train_encoded.shape[1]}")
print(f"Columnas en el conjunto de testeo: {test_data_encoded.shape[1]}")

# Busqueda de hiperparametros

## Random Forest

### Random search hiperparametros

In [9]:
# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'max_samples': np.linspace(0.5, 1.0, num=10),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Definir el modelo
model = RandomForestClassifier(random_state=random_state)

# Definir el RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=2,  # Número de combinaciones de hiperparámetros a probar
    scoring='roc_auc',
    cv=3,  # Número de folds en la validación cruzada
    verbose=2,
    random_state=random_state,
    n_jobs=-1  # Usar todos los núcleos disponibles
)



In [ ]:
# Ajustar el modelo a los datos de entrenamiento
random_search.fit(X_train_encoded, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

# Predecir en el conjunto de validación con el mejor modelo encontrado
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_val_encoded)[:, 1]

# Calcular el AUC en el conjunto de validación
auc = roc_auc_score(y_val, y_pred_proba)
print(f"AUC en el conjunto de validación: {auc:.4f}")

### Hyperopt

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros
space = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'max_depth': hp.choice('max_depth', [None, 10, 20, 30, 40, 50]),
    'max_samples': hp.uniform('max_samples', 0.5, 1.0),
    'min_samples_split': hp.choice('min_samples_split', [2, 5, 10]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 4]),
    'bootstrap': True
}

# Función objetivo para Hyperopt
def objective(params):
    # Asegurarse de que 'max_samples' solo se usa si 'bootstrap' es True
    if not params['bootstrap']:
        params['max_samples'] = None  # No se puede utilizar max_samples si bootstrap es False
    
    # Definir el modelo con los hiperparámetros actuales
    model = RandomForestClassifier(**params, random_state=random_state)
    
    # Entrenar el modelo directamente ya que el Target Encoder e Imputador ya se aplicaron
    model.fit(X_train_encoded, y_train)

    # Hacer predicciones en el conjunto de validación
    y_pred_proba = model.predict_proba(X_val_encoded)[:, 1]

    # Calcular el AUC en el conjunto de validación
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")

    # Retornar el valor negativo del AUC ya que Hyperopt minimiza por defecto
    return {'loss': 1-auc, 'status': STATUS_OK}

# Ejecutar la optimización
trials = Trials()  # Para guardar información sobre cada iteración
best = fmin(fn=objective, 
            space=space, 
            algo=tpe.suggest, 
            max_evals=10,  # Número de evaluaciones
            trials=trials)

print("Mejores hiperparámetros encontrados:")
print(best)

## XGBoost

### Random search hiperparametros

In [ ]:
# Definir el espacio de búsqueda de hiperparámetros para XGBoost
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 5, 10, 15, 20],  # Profundidad máxima del árbol
    'learning_rate': np.linspace(0.01, 0.3, 10),  # Tasa de aprendizaje
    'subsample': np.linspace(0.5, 1.0, num=5),  # Fracción de muestras a usar para cada árbol
    'colsample_bytree': np.linspace(0.5, 1.0, num=5),  # Fracción de características a usar para cada árbol
    'gamma': [0, 0.1, 0.2, 0.3],  # Regularización
    'reg_alpha': [0, 0.01, 0.1, 1],  # Regularización L1
    'reg_lambda': [1, 1.5, 2, 3]  # Regularización L2
}

# Definir el modelo
model = XGBClassifier(random_state=random_state, use_label_encoder=False, eval_metric='logloss')

# Definir el RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=2,  # Número de combinaciones de hiperparámetros a probar
    scoring='roc_auc',  # Métrica para evaluar
    cv=3,  # Número de folds en la validación cruzada
    verbose=2,
    random_state=random_state,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Ajustar el modelo a los datos de entrenamiento
random_search.fit(X_train_encoded, y_train)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

# Predecir en el conjunto de validación con el mejor modelo encontrado
best_model = random_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_val_encoded)[:, 1]

# Calcular el AUC en el conjunto de validación
auc = roc_auc_score(y_val, y_pred_proba)
print(f"AUC en el conjunto de validación: {auc:.4f}")

### Hyperopt

In [ ]:
# Espacio de búsqueda para los hiperparámetros
space = {
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400, 500]),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight': hp.uniform('min_child_weight', 1, 10),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0.5, 2)
}

# Función objetivo para Hyperopt
def objective(params):
    # Entrenar el modelo con los hiperparámetros actuales
    model = xgb.XGBClassifier(
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        n_estimators=int(params['n_estimators']),
        gamma=params['gamma'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        scale_pos_weight=params['scale_pos_weight'],
        use_label_encoder=False,
        eval_metric='auc',
        random_state=random_state
    )
    
    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train_encoded, y_train)
    
    # Predecir las probabilidades para calcular el AUC en el conjunto de validación
    y_pred_proba = model.predict_proba(X_val_encoded)[:, 1]
    auc = roc_auc_score(y_val, y_pred_proba)
    
    # Imprimir los hiperparámetros y el AUC actual
    print(f"Hiperparámetros: {params}, AUC: {auc:.4f}")
    
    return {'loss': -auc, 'status': STATUS_OK}

# Ejecutar la optimización
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=2,  # Número de evaluaciones
            trials=trials)

print("Mejores hiperparámetros encontrados:")
print(best)

In [ ]:
# Reentrenar el modelo con los mejores hiperparámetros
best_model = xgb.XGBClassifier(
    max_depth=int(best['max_depth']),
    learning_rate=best['learning_rate'],
    n_estimators=int(best['n_estimators']),
    gamma=best['gamma'],
    min_child_weight=best['min_child_weight'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    scale_pos_weight=best['scale_pos_weight'],
    use_label_encoder=False,
    eval_metric='auc',
    random_state=random_state
)

# Entrenar el modelo con los mejores hiperparámetros en los datos completos
best_model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de validación
y_pred_proba_val = best_model.predict_proba(X_val)[:, 1]
auc_val = roc_auc_score(y_val, y_pred_proba_val)
print(f"AUC en el conjunto de validación: {auc_val:.4f}")

## Pueba de Hiperparametros

### Random Forest

#### A mano

In [ ]:
# {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_samples': 0.6666666666666666, 'max_depth': 50, 'bootstrap': True}
# Reconstruir el modelo con los mejores hiperparámetros
best_model_rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=20,
    max_samples=None,
    min_samples_split=5,
    min_samples_leaf=4,
    bootstrap=False,
    random_state=random_state
)

# Crear el pipeline con el mejor modelo
rf_pipeline = make_pipeline(SimpleImputer(), best_model_rf)

# Entrenar el modelo con los datos de entrenamiento
rf_pipeline.fit(X_train_encoded, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_rf_test = rf_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"random_forest_predictions_n_estimators_{500}_"
    f"max_depth_{20}_"
    f"min_samples_split_{5}_"
    f"min_samples_leaf_{4}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")

#### Automatizada

In [ ]:
# Convertir los hiperparámetros a los tipos correctos
random_search.best_params_['n_estimators'] = int(random_search.best_params_['n_estimators'])
random_search.best_params_['min_samples_split'] = int(random_search.best_params_['min_samples_split'])
random_search.best_params_['min_samples_leaf'] = int(random_search.best_params_['min_samples_leaf'])
random_search.best_params_['max_depth'] = int(random_search.best_params_['max_depth']) if random_search.best_params_['max_depth'] is not None else None

# Reconstruir el modelo con los mejores hiperparámetros
best_model_rf = RandomForestClassifier(
    n_estimators=random_search.best_params_['n_estimators'],
    max_depth=random_search.best_params_['max_depth'],  # Asegurarse de que max_depth sea un entero o None
    min_samples_split=random_search.best_params_['min_samples_split'],
    min_samples_leaf=random_search.best_params_['min_samples_leaf'],
    bootstrap=random_search.best_params_['bootstrap'],
    random_state=random_state,
    max_samples=random_search.best_params_['max_samples']
)

# Crear el pipeline con el mejor modelo
rf_pipeline = make_pipeline(SimpleImputer(), best_model_rf)

# Entrenar el modelo con los datos de entrenamiento
rf_pipeline.fit(X_train_encoded, y_train)

In [ ]:
# Predecir en el conjunto de testeo
y_preds_rf_test = rf_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_rf = pd.DataFrame({"id": test_data["id"], "Label": y_preds_rf_test})
submission_df_rf["id"] = submission_df_rf["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "random_forest"
file_name_rf = (
    f"random_forest_predictions_n_estimators_{random_search.best_params_['n_estimators']}_"
    f"max_depth_{random_search.best_params_['max_depth']}_"
    f"min_samples_split_{random_search.best_params_['min_samples_split']}_"
    f"max_samples_{random_search.best_params_['max_samples']}_"
    f"bootstrap_{random_search.best_params_['bootstrap']}_"
    f"min_samples_leaf_{random_search.best_params_['min_samples_leaf']}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_rf.to_csv(os.path.join("submits", file_name_rf), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_rf}")

### XGboost

#### A mano

In [ ]:
# Reconstruir el modelo de XGBoost con los mejores hiperparámetros
best_model_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=3,
    learning_rate=0.016434700376417533,
    subsample=0.6696027253148403,
    scale_pos_weight=1.9885767576831004,
    colsample_bytree=0.6189520886008961,
    min_child_weight=6.378545393104531,
    gamma=0.05263130692482787,
    random_state=random_state,
    use_label_encoder=False,  # Si estás usando una versión reciente de XGBoost
    eval_metric="auc"  # Ajustar el métrico de evaluación
)

# Crear el pipeline con el mejor modelo
xgb_pipeline = make_pipeline(SimpleImputer(), best_model_xgb)

# Entrenar el modelo con los datos de entrenamiento
xgb_pipeline.fit(X_train_encoded, y_train)

# Predecir en el conjunto de testeo
y_preds_xgb_test = xgb_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_xgb = pd.DataFrame({"id": test_data["id"], "Label": y_preds_xgb_test})
submission_df_xgb["id"] = submission_df_xgb["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "xgboost"
file_name_xgb = (
    f"xgboost_predictions_n_estimators_{500}_"
    f"max_depth_{20}_"
    f"learning_rate_{0.05}_"
    f"min_child_weight_{4}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_xgb.to_csv(os.path.join("submits", file_name_xgb), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_xgb}")


#### Atomatizada

In [ ]:
#Hiperparámetros: {'colsample_bytree': 0.6813166881988515, 'gamma': 0.3836347732082877, 'learning_rate': 0.02522774305471077, 'max_depth': 4, 'min_child_weight': 9.181974700523355, 'n_estimators': 400, 'scale_pos_weight': 1.4846499378171216, 'subsample': 0.9484768092226656}, AUC: 0.8442

# Convertir los hiperparámetros a enteros si es necesario
best['n_estimators'] = int(best['n_estimators'])
best['max_depth'] = [None, 10, 20, 30, 40, 50][best['max_depth']]  # Ajustar el valor del índice si es necesario
best['scale_pos_weight'] = best['scale_pos_weight']

# Reconstruir el modelo de XGBoost con los mejores hiperparámetros
best_model_xgb = xgb.XGBClassifier(
    n_estimators=best['n_estimators'],
    max_depth=best['max_depth'],
    learning_rate=best['learning_rate'],
    subsample=best['subsample'],
    colsample_bytree=best['colsample_bytree'],
    min_child_weight=best['min_child_weight'],
    gamma=best['gamma'],
    random_state=random_state,
    use_label_encoder=False,
    scale_pos_weight=best['scale_pos_weight'],
    eval_metric="auc"  # Establecer AUC como métrica de evaluación
)

# Crear el pipeline con el mejor modelo
xgb_pipeline = make_pipeline(SimpleImputer(), best_model_xgb)

# Entrenar el modelo con los datos de entrenamiento
xgb_pipeline.fit(X_train_encoded, y_train)

# Predecir en el conjunto de testeo
y_preds_xgb_test = xgb_pipeline.predict_proba(test_data_encoded)[:, 1]

# Crear el archivo de envío
submission_df_xgb = pd.DataFrame({"id": test_data["id"], "Label": y_preds_xgb_test})
submission_df_xgb["id"] = submission_df_xgb["id"].astype(int)

# Crear el nombre del archivo basado en los mejores hiperparámetros, incluyendo "xgboost"
file_name_xgb = (
    f"xgboost_predictions_n_estimators_{best['n_estimators']}_"
    f"max_depth_{best['max_depth']}_"
    f"learning_rate_{best['learning_rate']}_"
    f"subsample_{best['subsample']}_"
    f"colsample_bytree_{best['colsample_bytree']}_"
    f"gamma_{best['gamma']}_"
    f"scale_pos_weight_{best['scale_pos_weight']}_"
    f"min_child_weight_{best['min_child_weight']}.csv"
)

# Crear la carpeta "submits" si no existe
os.makedirs("submits", exist_ok=True)

# Guardar el archivo de predicción en la carpeta 'submits'
submission_df_xgb.to_csv(os.path.join("submits", file_name_xgb), sep=",", index=False)

print(f"Archivo guardado en: submits/{file_name_xgb}")
